In [ ]:
import csv
import time

from tqdm import tqdm

from selenium import webdriver
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
)

In [ ]:
CHROME_OPTIONS = webdriver.ChromeOptions()
CHROME_OPTIONS.add_argument("--headless")

FILE = "/Users/alex.urban/Downloads/city-of-milwaukee-salaries-2020.tsv"

SOURCE = "https://assessments.milwaukee.gov/SearchResults.asp?SearchOwner"

In [ ]:
class wait_til_load(object):

    def __init__(self, f):
        self.f = f

    def __call__(self, *args):
        try:
            return self.f(*args)
        except (
            NoSuchElementException,
            StaleElementReferenceException,
        ):
            # retry with enough time to load the data
            return self.__call__(*args)


@wait_til_load
def _get_properties(browser):
    def parse_column(tbody, col):
        if col in [3, 5, 6, 7, 8]:
            # handle columns bafflingly packed with two entries
            packed = [row[col].text.split("\n")
                      for row in tbody]
            return ["\n".join(item) for item in
                    map(list, zip(*packed))]
        return "\n".join([row[col].text for row in tbody])

    if (
        "No matching records found" in
        browser.find_element_by_tag_name("body").text
    ):
        return list()
    # first, locate the table
    table = browser.find_element_by_xpath("//table[@id='T1']")
    # next, locate rows in the body of the table
    tbody = [item.find_elements_by_tag_name("td")
             for item in table.find_elements_by_tag_name("tr")[1::]]
    return [  # finally, parse these entries into CSV-friendly text
        parse_column(tbody, 0),  # taxkey
        parse_column(tbody, 1),  # address
        parse_column(tbody, 4),  # assessed value
    ] + (
        parse_column(tbody, 3) +  # build year, type
        parse_column(tbody, 5) +  # beds, baths
        parse_column(tbody, 6) +  # lot size, area (sq. ft.)
        parse_column(tbody, 7) +  # LUC, description
        parse_column(tbody, 8)    # sale date, sale price
    )

In [ ]:
# read in data from CSV
with open(FILE, "r") as fileobj:
    data = list(csv.reader(fileobj, delimiter="\t"))
header = data[0]
rows = data[1::]

In [ ]:
# connect via Chrome
browser = webdriver.Chrome(options=CHROME_OPTIONS)

In [ ]:
# get start time
start = time.time()

# read off table column headers
header += [
    "Property taxkey",
    "In-city property owned",
    "Assessed property value",
    "Build year",
    "Building type",
    "No. of beds",
    "No. of baths",
    "Lot size (sq. ft.)",
    "Floor area (sq. ft.)",
    "LUC",
    "Description",
    "Sale date",
    "Sale price",
]

# retrieve data from the table
for i in tqdm(range(len(rows)), desc="Progress: "):
    if rows[i][0].strip():
        name = "+".join(rows[i][0].split(",")[::-1]).replace(" ", "+")
        browser.get(f"{SOURCE}={name}")
        try:
            rows[i] += _get_properties(browser)
        except RecursionError:
            continue

# report the time taken
minutes = (time.time() - start) / 60
print(f"Total time taken: {minutes} minutes")

In [ ]:
# close the browser
browser.close()

In [ ]:
# overwrite the original file
with open(FILE, "w") as fileobj:
    writer = csv.writer(fileobj, delimiter="\t")
    writer.writerow(header)  # column headers
    writer.writerows(rows)  # column data